In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])

In [3]:
#Sort values
sort_prices = prices.sort_values('updated_at')
sort_sales = sales.sort_values('ordered_at')

In [4]:
#Merge sales table & prices table
combine_backward = pd.merge_asof(sort_sales, sort_prices, by = 'product_id', left_on = ['ordered_at'], right_on = ['updated_at'], direction = 'backward')
combine_backward['final_price'] = combine_backward['new_price']
combine_forward = pd.merge_asof(sort_sales, sort_prices, by = 'product_id', left_on = ['ordered_at'], right_on = ['updated_at'], direction = 'forward')
combine_forward['final_price'] = combine_forward['old_price']

In [5]:
#Remove null values in combine_backward
drop_na_bw = combine_backward.dropna()

In [6]:
#Filter values from combine_foward
combine_fw_drop_bw = combine_forward.merge(drop_na_bw, on = ['product_id', 'ordered_at'], how = 'left', indicator = True)
drop_list = combine_fw_drop_bw.loc[combine_fw_drop_bw['_merge'] == 'left_only','ordered_at']
add_value = combine_forward[combine_forward['ordered_at'].isin(drop_list)]

In [7]:
#Union tables
combine = pd.concat([drop_na_bw, add_value], ignore_index = True)
combine['revenue'] = combine['final_price'] * combine['quantity_ordered']
get_data = combine.groupby(['product_id','final_price']).agg({'revenue':'sum'})
result = get_data.groupby('product_id').agg({'revenue':'sum'})
print(result.reset_index())

   product_id    revenue
0          64   956000.0
1     3954203   877500.0
2     3998909   280500.0
3     4085861  8247500.0
